In [1]:
%pip install --upgrade llama-index        # core
%pip install --upgrade llama-index-llms-google-genai  # Google / Gemini LLM integration
%pip install --upgrade llama-index-embeddings-google-genai  # embeddings via Google GenAI
%pip install --upgrade google-generativeai  # underlying Google SDK
%pip install --upgrade pinecone-client  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# ---------------------------- IMPORTS (all in one place)
from textwrap import shorten
from llama_index.core import (Settings,Document,VectorStoreIndex,SummaryIndex,KeywordTableIndex,TreeIndex,)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [6]:
# ============================ Embedding layer (no API calls)
#============================
Settings.embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")
Settings.llm = None # ensure NO generation; we'll only retrieve Nodes
# =========

LLM is explicitly disabled. Using MockLLM.


In [7]:
# ============================ Data (sample texts → Documents; no manual Nodes) ============================
docs = [
 Document(text="Turmeric coffee (golden latte) blends coffee with turmeric and spices; earthy and slightly peppery."),
 Document(text="Ashwagandha often appears in wellness menus; exact keyword lookups help when scanning titles and headings."),
 Document(text="Brewing temperature: lighter roasts typically 92-96°C; hotter extractions risk bitterness."),
 Document(text="Espresso uses fine grind, ~9 bar pressure, and short contact time to create a concentrated cup."),
]   

In [8]:
# ============================ Index build (LlamaIndex will create Nodes internally) ============================
vec_index = VectorStoreIndex.from_documents(docs,transformations=[]) # semantic search
sum_index = SummaryIndex.from_documents(docs, transformations=[])# precomputed summaries
kw_index = KeywordTableIndex.from_documents(docs,transformations=[]) # keyword table
tree_index = TreeIndex.from_documents(docs, transformations=[])# hierarchical
# ============================ "Show me a Node" -- WITHOUT creating one explicitly ============================

In [9]:
# ============================ "Show me a Node" -- WITHOUT creating one explicitly ============================
def peek_one_node(index, query: str):
 hit = index.as_retriever(similarity_top_k=1).retrieve(query)[0]
 node = hit.node
 node_id = getattr(node, "node_id", getattr(node, "id_", None))
 print(f"\n[{index.__class__.__name__}] query='{query}'")
 print("node_id :", node_id)
 print("text :", shorten(node.get_content(), width=140,
placeholder=" ..."))
 print("metadata:", dict(list(node.metadata.items())[:4]))
 rels = getattr(node, "relationships", {})
 print("rels :", list(rels.keys()) if rels else "--")

In [10]:
# VectorStoreIndex → returns the embedded Node that matched semantically
peek_one_node(vec_index, "What is turmeric coffee?")
# SummaryIndex → returns the summary Node it stored for the doc
peek_one_node(sum_index, "Explain turmeric coffee in brief")
# KeywordTableIndex → returns a Node found via exact term lookup
peek_one_node(kw_index, "ashwagandha")
# TreeIndex → returns a Node from the most relevant branch(top→down)
peek_one_node(tree_index, "brewing temperature")


[VectorStoreIndex] query='What is turmeric coffee?'
node_id : 1daa23ea-7aa1-452e-b91e-241ac19ae76a
text : Turmeric coffee (golden latte) blends coffee with turmeric and spices; earthy and slightly peppery.
metadata: {}
rels : [<NodeRelationship.SOURCE: '1'>]

[SummaryIndex] query='Explain turmeric coffee in brief'
node_id : d37b06ec-0f7a-4285-abd5-a72869ed64d8
text : Turmeric coffee (golden latte) blends coffee with turmeric and spices; earthy and slightly peppery.
metadata: {}
rels : [<NodeRelationship.SOURCE: '1'>]

[KeywordTableIndex] query='ashwagandha'
node_id : 151eecde-781d-4ffe-b195-f503f2614e7b
text : Ashwagandha often appears in wellness menus; exact keyword lookups help when scanning titles and headings.
metadata: {}
rels : [<NodeRelationship.SOURCE: '1'>]

[TreeIndex] query='brewing temperature'
node_id : 64e4f599-a326-4904-a336-7bd0ae9827a4
text : Turmeric coffee (golden latte) blends coffee with turmeric and spices; earthy and slightly peppery.
metadata: {}
rels : [<Node